In [5]:
# Import necessary libraries
import os
import h5py
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
from skimage.transform import resize
import tensorflow.keras.backend as K
import datetime
import json
import matplotlib.pyplot as plt

# -------------------------------
# 1. Verify GPU Availability
# -------------------------------
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Available GPU devices:", tf.config.list_physical_devices('GPU'))

# -------------------------------
# 2. Define Custom Metrics
# -------------------------------
def dice_coefficient(y_true, y_pred, smooth=1):
    """
    Calculates the Dice Coefficient.
    """
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def iou_metric(y_true, y_pred, smooth=1):
    """
    Calculates the Intersection over Union (IoU) metric.
    """
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

Num GPUs Available:  0
Available GPU devices: []


In [6]:
# -------------------------------
# 3. Define the U-Net Model
# -------------------------------
def unet_model(input_size=(128, 128, 4)):
    
    inputs = Input(input_size)

    # Encoding path
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Decoding path
    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    return 

In [7]:
def load_data(train_files, data_dir, image_dim=(128, 128)):
    """
    Function to load images and masks from HDF5 files.

    Parameters:
    - train_files: List of file names of the training HDF5 files.
    - data_dir: Directory where the HDF5 files are stored.
    - image_dim: Desired dimensions of the output images and masks (height, width).

    Returns:
    - X: Array of image data with shape (batch_size, height, width, channels).
    - y: Array of mask data with shape (batch_size, height, width, 1).
    """
    batch_size = len(train_files)

    # Initialize empty arrays for images and masks
    X = np.empty((batch_size, *image_dim, 4), dtype='float32')  # 4 channels for images
    y = np.empty((batch_size, *image_dim, 1), dtype='float32')  # Single-channel masks
    
    # Loop through each HDF5 file
    for i, h5_file in enumerate(train_files):
        file_path = os.path.join(data_dir, h5_file)
        with h5py.File(file_path, 'r') as hf:
            # Check for the correct keys
            if 'image' in hf.keys() and 'mask' in hf.keys():
                image = hf['image'][:]  # Expected shape: (H, W, 4)
                mask = hf['mask'][:]    # Expected shape: (H, W) or (H, W, C)
            else:
                raise KeyError(f"Unexpected keys in {h5_file}: {list(hf.keys())}")

            # If mask has multiple channels, convert to single-channel
            if mask.ndim > 2:
                mask = np.mean(mask, axis=-1)  # Convert RGB to grayscale

            # Resize images if necessary
            if image.shape[:2] != image_dim:
                image = resize(
                    image, 
                    (*image_dim, image.shape[2]), 
                    preserve_range=True, 
                    anti_aliasing=True
                )

            if mask.shape != image_dim:
                # Use nearest-neighbor interpolation for masks to preserve labels
                mask = resize(
                    mask, 
                    image_dim, 
                    preserve_range=True, 
                    order=0, 
                    anti_aliasing=False
                )

            # Normalize the image
            image_max = np.max(image)
            if image_max > 0:
                image = image.astype('float32') / image_max
            else:
                image = image.astype('float32')

            # Normalize the mask
            mask_max = np.max(mask)
            if mask_max > 0:
                mask = mask.astype('float32') / mask_max
            else:
                mask = mask.astype('float32')  # All zeros

            # Assign to batch arrays
            X[i] = image  # Shape: (128, 128, 4)
            y[i] = np.expand_dims(mask, axis=-1)  # Shape: (128, 128, 1)
    
    return X, y

In [8]:
# -------------------------------
# 4. Prepare the Data
# -------------------------------

# Define the data directory (adjust this path if necessary)
data_dir = 'BraTS2020_training_data/content/small_data/'

# Get list of .h5 files
h5_files = [f for f in os.listdir(data_dir) if f.endswith('.h5')]

# Ensure that there are .h5 files in the directory
if not h5_files:
    raise FileNotFoundError(f"No .h5 files found in the directory: {data_dir}")

# Split into training and validation sets
train_files, val_files = train_test_split(h5_files, test_size=0.2, random_state=42)

print(f"Total samples: {len(h5_files)}")
print(f"Training samples: {len(train_files)}")
print(f"Validation samples: {len(val_files)}")

# Parameters
batch_size = 8  # Adjust based on GPU memory
image_dim = (128, 128)  # Adjust based on your image dimensions

X_train, y_train = load_data(train_files, data_dir, image_dim=(128, 128))

X_val, y_val = load_data(val_files, data_dir, image_dim=(128, 128))

# -------------------------------
# 7. Build and Compile the Model
# -------------------------------
model = unet_model(input_size=(*image_dim, 4))  # Updated input size
model.compile(optimizer=Adam(), 
              loss='binary_crossentropy', 
              metrics=['accuracy', dice_coefficient, iou_metric])
model.summary()

# -------------------------------
# 8. Define Callbacks
# -------------------------------
log_dir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

# TensorBoard callback for visualization
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# EarlyStopping callback with multiple metric monitoring
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    verbose=1, 
    restore_best_weights=True
)

# ModelCheckpoint callback to save the best model with '.keras' extension
model_checkpoint = ModelCheckpoint(
    'model-unet.best.keras',  # Changed from '.h5' to '.keras'
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True, 
    mode='min'
)

# Combine all callbacks
callbacks = [
    early_stopping,
    model_checkpoint,
    tensorboard_callback
]

# -------------------------------
# 9. Load the Data
# -------------------------------
# Assuming you have the DataGenerator instances defined for training and validation
# If you're using X_train and y_train, manually fetch the first batch from the generators:

# -------------------------------
# 10. Train the Model with X_train and y_train
# -------------------------------
epochs = 3  # You can adjust this based on your requirements

try:
    results = model.fit(
        X_train, y_train,  # Use X_train and y_train directly
        validation_data=(X_val, y_val),  # Use X_val and y_val directly
        epochs=epochs,
        callbacks=callbacks,
        verbose=1  # Set to 1 for progress bar, 2 for one line per epoch, 0 for silent
    )
except Exception as e:
    print(f"An error occurred during training: {e}")

Total samples: 62
Training samples: 49
Validation samples: 13


AttributeError: 'NoneType' object has no attribute 'compile'